# Test to see if 2 summary prompts are equivalent

In [25]:
import requests
import json
import os

def sendchat(instructions):
    API_KEY = "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"
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    payload = {
        "model": "openai/gpt-4.1",
        "messages": [
            {
                "role": "user",
                "content": instructions,
            },
        ],
        "stream": False,
    }

    response = requests.post(url, headers=headers, json=payload)
    return response.json()

In [ ]:
opinions = ["Current abortion laws are too laxed, it should be illegal in all states. no children should be killed.",
"I think everyone should bear arms freely. we should fight for our rights and protect ourselves. fuck the democrats",
"there should be more promotion opportunities for minorities and women in the workplace. Maternal leave should be longer and mandatory for all companies",
"There should be stricter gun control measures. It's crazy that in the most developed country in the world we still have school shooting."]


In [21]:
responses = {}
for i, opinion in enumerate(opinions):
    prompt1 = f"""Create a one-sentence summary of the following argument.

    - Omit the reasons provided (e.g., "abortion should be legal because it is a right" → "abortion should be legal")
    - Avoid double-barreled questions (only one issue position and one concept at a time)
    - If multiple issues are mentioned, focus on the first
    - Stick to what the person says
    - Write it from the first person ("I believe ...")
    - Remove everything after "because"
    - Keep it under 15 words
    - Return only one statement (not "I believe X and Y", only "I believe X")

    Argument: {opinion}
    Summary:"""

    prompt2 = f"""Create a one-sentence summary of the following argument. The summary should start with "I believe" and only retain one clear position about one main issue from the original argument.

    Argument: {opinion}
    Summary:"""

    for n, prompt in {"prompt1": prompt1, "prompt2": prompt2}.items():
        response = sendchat(prompt)
        responses[f"{n}_{i}"] = response

In [22]:
responses['prompt1_1']

{'id': 'gen-1757007136-MM30bi11c2KbVZGX1RRm',
 'provider': 'OpenAI',
 'model': 'openai/gpt-4.1',
 'object': 'chat.completion',
 'created': 1757007136,
 'choices': [{'logprobs': None,
   'finish_reason': 'stop',
   'native_finish_reason': 'stop',
   'index': 0,
   'message': {'role': 'assistant',
    'content': 'I believe everyone should bear arms freely.',
    'refusal': None,
    'reasoning': None}}],
 'system_fingerprint': 'fp_3502f4eb73',
 'usage': {'prompt_tokens': 171,
  'completion_tokens': 8,
  'total_tokens': 179,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0}}}

In [23]:
for n, response in responses.items():
    print(n, response["choices"][0]["message"])

prompt1_0 {'role': 'assistant', 'content': 'I believe abortion should be illegal in all states.', 'refusal': None, 'reasoning': None}
prompt2_0 {'role': 'assistant', 'content': 'I believe abortion should be illegal in all states.', 'refusal': None, 'reasoning': None}
prompt1_1 {'role': 'assistant', 'content': 'I believe everyone should bear arms freely.', 'refusal': None, 'reasoning': None}
prompt2_1 {'role': 'assistant', 'content': 'I believe everyone should have the right to bear arms.', 'refusal': None, 'reasoning': None}
prompt1_2 {'role': 'assistant', 'content': 'I believe there should be more promotion opportunities for minorities and women in the workplace.', 'refusal': None, 'reasoning': None}
prompt2_2 {'role': 'assistant', 'content': 'I believe there should be more promotion opportunities for minorities and women in the workplace.', 'refusal': None, 'reasoning': None}


## They seem to yield similar results -> change summary block.

In [ ]:
opinions={
    "abortion": "over-the-counter pills should be easily available for women seeking abortion. Everyone should have the right to abort legally",
    "lgbtq+": "there should be anti-discrimination laws against businesses that refuse to serve LGBTQ+ communities. Love is love, non-heterosexual relationships should be protected by law.",
    "foreign policy": "the U.S should reduce funds in overseas war and focus funding on more science programs and infrastructure.",
    "climate change": "anthropogenic climate change is real and we should take action to reduce greenhouse gas emissions. There should be stricter regulations on carbon emissions and fossil fuel subsidies."
}
responses = {}
for topic, opinion in opinions.items():
    prompt1 = f"""Create a one-sentence summary of the following argument.

    - Omit the reasons provided (e.g., "abortion should be legal because it is a right" → "abortion should be legal")
    - Avoid double-barreled questions (only one issue position and one concept at a time)
    - If multiple issues are mentioned, focus on the first
    - Stick to what the person says
    - Write it from the first person ("I believe ...")
    - Remove everything after "because"
    - Keep it under 15 words
    - Return only one statement (not "I believe X and Y", only "I believe X")

    Argument: {opinion}
    Summary:"""

    prompt2 = f"""Please infer the topic and the position regarding the topic of the following argument, denoted by <argument:>, choosing from the following options, which are delimited by a semicolon: very conserrvative; somewhat conservative; slightly conservative; moderate; slightly liberal; somewhat liberal; very liberal.
    Create a one-sentence summary of the argument. The summary should start with "I believe" and only retain a clear position about the main political issue from the original argument.
    Answer in the following format:
    <topic:> <position:>
    <Ssmmary:>
    <argument:> {opinion}
    """

    prompt3 = f"""Infer the main political topic and the stance in the following argument (<argument:>).
    Choose the stance from this list: very conservative; somewhat conservative; slightly conservative; moderate; slightly liberal; somewhat liberal; very liberal.

    Then:
    1. Identify the main political topic.
    2. Assign the stance from the list above.
    3. Write a one-sentence summary that starts with "I believe" and expresses only the main position regarding the issue from the original argument.

    Format the answer exactly like this:
    <topic:> <position:>
    <summary:> ...
    <argument:> {opinion}
    """

    prompt4 = f"""<argument:> {opinion}.
    Infer the stance of the above argument (<argument:>) about {topic}.
    Choose the stance from this list: very conservative; somewhat conservative; slightly conservative; moderate; slightly liberal; somewhat liberal; very liberal. 
    Then, write a one-sentence summary that starts with "I believe" that includes only the main position from the original argument.
    Format the answer exactly like this:
    <position:>
    <summary:> 
    """

    prompt5 = f"""<argument:> {opinion}.
    Infer the stance of the above argument (<argument:>) about {topic}; choose from this list: very conservative, somewhat conservative, slightly conservative, moderate, slightly liberal, somewhat liberal, very liberal. 
    Write a one-sentence summary that starts with "I believe"; only express a clear position regarding the issue.
    Format the answer exactly like this:
    <position:>
    <summary:> 
    """

    prompt6 = f"""<argument:> {opinion}.
    Infer the stance of the above argument (<argument:>) about the topic of {topic}. Choose the stance from this list: very conservative, somewhat conservative, slightly conservative, moderate, slightly liberal, somewhat liberal, very liberal. 
    Create a one-sentence summary of the argument. The summary should start with "I believe" and only retain one main clear position on the topic.
    <position:>
    <summary:> 
    """

    prompt7 = f"""<argument:> {opinion}.
    Infer the stance of the above argument (<argument:>) about the topic of {topic}. Choose the stance from this list: very conservative, somewhat conservative, slightly conservative, moderate, slightly liberal, somewhat liberal, very liberal. 
    Create a one-sentence summary of the argument. The summary should start with "I believe" and only express one concept regarding the issue at a time, ignore the rest of the argument if needed. 
    Format the answer as follows:
    <position:>
    <summary:> 
    """ 

    prompt8 = f"""<argument:> {opinion}.
    Create a one-sentence summary of the argument above argument (<argument:>) about the topic of {topic}. The summary should start with "I believe" and only express one concept regarding the issue at a time, ignore the rest of the argument if needed. 
    Infer the stance of the argument, choose from this list: very conservative, somewhat conservative, slightly conservative, moderate, slightly liberal, somewhat liberal, very liberal. 
    <position:>
    <summary:> 
    """

    # for n, prompt in {"prompt2": prompt2, "prompt3": prompt3}.items():
    for n, prompt in {"prompt7": prompt7, "prompt8": prompt8}.items():
        response = sendchat(prompt)
        responses[f"{n}_{topic}"] = response
for n, response in responses.items():
    print(n, response["choices"][0]["message"])
    

prompt7_abortion {'role': 'assistant', 'content': '<position:> very liberal  \n<summary:> I believe that abortion should be a legal right for everyone.', 'refusal': None, 'reasoning': None}
prompt8_abortion {'role': 'assistant', 'content': '<position:> very liberal  \n<summary:> I believe women should have the legal right to access abortion.', 'refusal': None, 'reasoning': None}
prompt7_lgbtq+ {'role': 'assistant', 'content': '<position:> very liberal  \n<summary:> I believe anti-discrimination laws should protect LGBTQ+ people from being refused service by businesses.', 'refusal': None, 'reasoning': None}
prompt8_lgbtq+ {'role': 'assistant', 'content': '<position:> very liberal  \n<summary:> I believe businesses should be legally prohibited from discriminating against LGBTQ+ individuals.', 'refusal': None, 'reasoning': None}
prompt7_foreign policy {'role': 'assistant', 'content': '<position:> somewhat liberal  \n<summary:> I believe the U.S. should reduce its military spending oversea

In [43]:
responses['prompt7_abortion']["choices"][0]["message"]

{'role': 'assistant',
 'content': '<position:> very liberal  \n<summary:> I believe that abortion should be a legal right for everyone.',
 'refusal': None,
 'reasoning': None}

In [45]:
# parse the response
# get the position and summary
res = responses['prompt7_abortion']["choices"][0]["message"]['content']
position = res.split(" \n<summary:> ")[0].split("<position:> ")[1]
summary = res.split(" \n<summary:> ")[1]

print('position: ', position)
print('summary: ', summary)

position:  very liberal 
summary:  I believe that abortion should be a legal right for everyone.


## -> Prompt 7 looks good

## These prompts still give too long of an answer 

prompt6_abortion {'role': 'assistant', 'content': '<position:> very liberal  \n<summary:> I believe abortion should be easily accessible and legal for everyone who seeks it.', 'refusal': None, 'reasoning': None}

prompt6_lgbtq+ {'role': 'assistant', 'content': '<position:> very liberal  \n<summary:> I believe there should be anti-discrimination laws to protect LGBTQ+ individuals from being refused service by businesses.', 'refusal': None, 'reasoning': None}

prompt6_foreign policy {'role': 'assistant', 'content': '<position:> somewhat liberal  \n<summary:> I believe the U.S. should shift funding away from overseas wars toward domestic science and infrastructure programs.', 'refusal': None, 'reasoning': None}

prompt6_climate change {'role': 'assistant', 'content': '<position:> somewhat liberal\n\n<summary:> I believe that humans are causing climate change and that we need stricter regulations to reduce greenhouse gas emissions.', 'refusal': None, 'reasoning': None}

prompt5_abortion {'role': 'assistant', 'content': '<position:> very liberal\n<summary:> I believe that abortion should be easily accessible and legal for everyone who chooses it.', 'refusal': None, 'reasoning': None}

prompt5_lgbtq+ {'role': 'assistant', 'content': '<position:> very liberal  \n<summary:> I believe that LGBTQ+ individuals should be fully protected by anti-discrimination laws and that businesses should not be allowed to refuse service based on sexual orientation or gender identity.', 'refusal': None, 'reasoning': None}

prompt5_foreign policy {'role': 'assistant', 'content': '<position:> somewhat liberal  \n<summary:> I believe the U.S. should redirect funds from overseas military engagements to invest more in scientific research and infrastructure at home.', 'refusal': None, 'reasoning': None}

prompt5_climate change {'role': 'assistant', 'content': '<position:> very liberal  \n<summary:> I believe climate change is caused by human activity and we need strong regulatory action to reduce carbon emissions and end fossil fuel subsidies.', 'refusal': None, 'reasoning': None}

## Compare prompt1 vs 4

prompt1_abortion {'role': 'assistant', 'content': 'I believe over-the-counter pills should be easily available for women seeking abortion.', 'refusal': None, 'reasoning': None}

prompt1_lgbtq+ {'role': 'assistant', 'content': 'I believe there should be anti-discrimination laws against businesses that refuse to serve LGBTQ+ communities.', 'refusal': None, 'reasoning': None}

prompt1_foreign policy {'role': 'assistant', 'content': 'I believe the U.S should reduce funds in overseas war.', 'refusal': None, 'reasoning': None}

prompt1_climate change {'role': 'assistant', 'content': 'I believe anthropogenic climate change is real.', 'refusal': None, 'reasoning': None}

prompt4_abortion {'role': 'assistant', 'content': '<position:> very liberal  \n<summary:> I believe women should have easy and legal access to over-the-counter abortion pills and the right to abort.', 'refusal': None, 'reasoning': None}

prompt4_lgbtq+ {'role': 'assistant', 'content': '<position:> very liberal  \n<summary:> I believe there should be anti-discrimination laws protecting LGBTQ+ individuals and their relationships.', 'refusal': None, 'reasoning': None}

prompt4_foreign policy {'role': 'assistant', 'content': '<position:> somewhat liberal  \n<summary:> I believe the U.S. should reduce military spending overseas and instead allocate more funds to science programs and domestic infrastructure.', 'refusal': None, 'reasoning': None}

prompt4_climate change {'role': 'assistant', 'content': '<position:> very liberal\n<summary:> I believe anthropogenic climate change is real and we should implement stricter regulations to reduce greenhouse gas emissions.', 'refusal': None, 'reasoning': None}

## Compare different versions of prompt 2

prompt2_0 {'role': 'assistant', 'content': '<topic:> Abortion rights; very liberal  \n<Summary:> I believe that abortion should be accessible and legal for all women.  \n<argument:> over-the-counter pills should be easily available for women seeking abortion. Everyone should have the right to abort legally', 'refusal': None, 'reasoning': None}

prompt2_1 {'role': 'assistant', 'content': '<topic:> LGBTQ+ rights/anti-discrimination laws; somewhat liberal  \n<Summary:> I believe there should be legal protections against discrimination for LGBTQ+ individuals by businesses.', 'refusal': None, 'reasoning': None}

prompt3_0 {'role': 'assistant', 'content': '<topic:> Abortion rights: very liberal  \n<summary:> I believe abortion should be legal and over-the-counter pills should be easily available to women.', 'refusal': None, 'reasoning': None}

prompt3_1 {'role': 'assistant', 'content': '<topic:> LGBTQ+ rights: somewhat liberal  \n<summary:> I believe businesses should be legally prohibited from discriminating against LGBTQ+ individuals.', 'refusal': None, 'reasoning': None}

prompt3_0 {'role': 'assistant', 'content': '<topic:> Government spending priorities : somewhat liberal  \n<summary:> I believe the U.S. should reduce military spending abroad and invest more in science and infrastructure at home.', 'refusal': None, 'reasoning': None}
